In [3]:
import tensorflow as tf
from transformers import BertTokenizer
import numpy as np
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Attention, Input, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

import sys
sys.path.append('pizza/utils')

2024-12-20 22:56:37.643463: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734728197.659941   38782 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734728197.664928   38782 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 22:56:37.680956: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/media/mohamed/AC6030326030059C/CMP1Materials/Forth/First/NLP/Project/dataset/venv/lib/python3.12/site-packages/tqdm/auto.py:

In [7]:
import numpy as np
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import json

from sklearn.model_selection import train_test_split

def create_encoder_decoder_model(bert_model, hidden_dim, num_labels_pizza, num_labels_drinks, max_length):
    # Define BERT input layers
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
    attention_mask_pizza = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_pizza")
    attention_mask_drinks = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_drinks")

    for layer in bert_model.layers:
        layer.trainable = False
    # BERT output
    bert_output = bert_model(input_ids=input_ids, attention_mask=attention_mask_pizza)
    bert_embeddings = bert_output.last_hidden_state  # Shape: (batch_size, seq_len, hidden_dim)

    # Add Bidirectional LSTM layers
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_dim, return_sequences=True))(bert_embeddings)
    x = tf.keras.layers.LSTM(hidden_dim, return_sequences=True)(x)

    # Add dropout and regularization to decoder LSTM
    decoder_lstm = tf.keras.layers.LSTM(256, return_sequences=True,
                       dropout=0.2, recurrent_dropout=0.2,
                       kernel_regularizer=tf.keras.regularizers.l2(0.01),
                       name="Decoder_LSTM")
    decoder_outputs = decoder_lstm(x)

    # Project both tensors to the same dimension (256)
    x_projected = tf.keras.layers.Dense(256)(x)
    attention_pizza = tf.keras.layers.Attention(name="Attention_Layer_Pizza")([decoder_outputs, x_projected])
    attention_drinks = tf.keras.layers.Attention(name="Attention_Layer_Drinks")([decoder_outputs, x_projected])
    
    # Add dropout after attention
    attention_pizza = tf.keras.layers.Dropout(0.2)(attention_pizza)
    attention_drinks = tf.keras.layers.Dropout(0.2)(attention_drinks)

    combined_pizza = tf.keras.layers.Concatenate()([decoder_outputs, attention_pizza])
    combined_drinks = tf.keras.layers.Concatenate()([decoder_outputs, attention_drinks])

    # Add batch normalization and dropout before final layers
    combined_pizza = tf.keras.layers.BatchNormalization()(combined_pizza)
    combined_pizza = tf.keras.layers.Dropout(0.2)(combined_pizza)
    combined_drinks = tf.keras.layers.BatchNormalization()(combined_drinks)
    combined_drinks = tf.keras.layers.Dropout(0.2)(combined_drinks)

    # Pizza Output
    pizza_output = tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(num_labels_pizza, 
                              activation="softmax", 
                              kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        name="Pizza_Output_Layer"
    )(combined_pizza)

    # Drinks Output
    drinks_output = tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(num_labels_drinks, 
                              activation="softmax", 
                              kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        name="Drinks_Output_Layer"
    )(combined_drinks)

    # Define the model with two outputs
    model = tf.keras.Model(inputs=[input_ids, attention_mask_pizza, attention_mask_drinks], outputs=[pizza_output, drinks_output], name="Hybrid_Encoder_Decoder_NER")

    return model

    
    
vocab_size = 30522  # this is the size of the BERT tokenizer
embedding_dim = 128
hidden_dim = 64
num_labels = 33
max_len=30 # max length of the input sequences is 25


# entities=np.load('full_negate_entities.npy', allow_pickle=True)
# entities_id = {e.item(): i+1 for i, e in enumerate(entities)}
# entities_id['0']=0
# entities_id['O']=0

# reversed_entities_id = {v: k for k, v in entities_id.items() if k != 0}
# reversed_entities_id[0]='O'

# print(entities_id)
# print(reversed_entities_id)


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

model = create_encoder_decoder_model(bert_model, hidden_dim, num_labels_pizza=21,num_labels_drinks=21, max_length=max_len)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Exception encountered when calling layer 'tf_bert_model_2' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tf_bert_model_2' (type TFBertModel):
  • input_ids=<KerasTensor shape=(None, 30), dtype=int32, sparse=False, name=input_ids>
  • attention_mask=<KerasTensor shape=(None, 30), dtype=int32, sparse=False, name=attention_mask_pizza>
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

In [4]:
def intialize_model(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN, NUM_CLASSES_PIZZA, NUM_CLASSES_DRINKS):
    input_seq = tf.keras.layers.Input(shape=(MAX_LEN,), dtype='int32', name="Input_Sequence")

    # Add dropout to embedding layer
    embedding = tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_LEN, name="Embedding_Layer")(input_seq)
    embedding = tf.keras.layers.Dropout(0.2)(embedding)
    
    # Add Gaussian noise layer
    noisy_embedding = tf.keras.layers.GaussianNoise(0.1)(embedding)

    # Add recurrent dropout to LSTM
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(128, return_sequences=True, return_state=True, 
             dropout=0.2, recurrent_dropout=0.2,
             kernel_regularizer=tf.keras.regularizers.l2(0.01),
             name="Encoder_LSTM"),
        name="Bidirectional_LSTM"
    )(noisy_embedding)  # Using noisy embedding instead of regular embedding

    state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])
    state_c = tf.keras.layers.Concatenate()([forward_c, backward_c])

    # Rest of the model remains the same
    decoder_lstm = tf.keras.layers.LSTM(256, return_sequences=True,
                       dropout=0.2, recurrent_dropout=0.2,
                       kernel_regularizer=tf.keras.regularizers.l2(0.01),
                       name="Decoder_LSTM")
    decoder_outputs = decoder_lstm(encoder_outputs, initial_state=[state_h, state_c])

    attention = tf.keras.layers.Attention(name="Attention_Layer")([decoder_outputs, encoder_outputs])
    attention = tf.keras.layers.Dropout(0.2)(attention)

    combined = tf.keras.layers.Concatenate()([decoder_outputs, attention])
    combined = tf.keras.layers.BatchNormalization()(combined)
    combined = tf.keras.layers.Dropout(0.2)(combined)

    pizza_output = tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(NUM_CLASSES_PIZZA, 
                              activation="softmax", 
                              kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        name="Pizza_Output_Layer"
    )(combined)

    drinks_output = tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(NUM_CLASSES_DRINKS, 
                              activation="softmax", 
                              kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        name="Drinks_Output_Layer"
    )(combined)

    model = tf.keras.Model(inputs=input_seq, outputs=[pizza_output, drinks_output], name="Hybrid_Encoder_Decoder_NER")

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0),
                  loss={
                      "Pizza_Output_Layer": "sparse_categorical_crossentropy",
                      "Drinks_Output_Layer": "sparse_categorical_crossentropy"
                  },
                  metrics={
                      "Pizza_Output_Layer": "accuracy",
                      "Drinks_Output_Layer": "accuracy"
                  })

    model.summary()

    return model


VOCAB_SIZE = 30522
EMBEDDING_DIM = 128  
MAX_LEN = 30
NUM_CLASSES_PIZZA = 21
NUM_CLASSES_DRINKS = 21

model= intialize_model(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN, NUM_CLASSES_PIZZA,NUM_CLASSES_DRINKS)

/media/mohamed/AC6030326030059C/CMP1Materials/Forth/First/NLP/Project/dataset/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1734728203.916194   38782 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1649 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "Hybrid_Encoder_Decoder_NER"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_Sequence      │ (None, 30)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Embedding_Layer     │ (None, 30, 128)   │  3,906,816 │ Input_Sequence[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 30, 128)   │          0 │ Embedding_Layer[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gaussian_noise      │ (None, 30, 128)   │          0 │ dropout[0][0]     │
│ (GaussianNoise)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Bidirectional_LSTM  │ [(None, 30, 256), │    263,168 │ gaussian_noise[0… │
│ (Bidirectional)     │ (None, 128),      │            │                   │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ Bidirectional_LS… │
│ (Concatenate)       │                   │            │ Bidirectional_LS… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256)       │          0 │ Bidirectional_LS… │
│ (Concatenate)       │                   │            │ Bidirectional_LS… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Decoder_LSTM (LSTM) │ (None, 30, 256)   │    525,312 │ Bidirectional_LS… │
│                     │                   │            │ concatenate[0][0… │
│                     │                   │            │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Attention_Layer     │ (None, 30, 256)   │          0 │ Decoder_LSTM[0][… │
│ (Attention)         │                   │            │ Bidirectional_LS… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 30, 256)   │          0 │ Attention_Layer[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 30, 512)   │          0 │ Decoder_LSTM[0][… │
│ (Concatenate)       │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 30, 512)   │      2,048 │ concatenate_2[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 30, 512)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Pizza_Output_Layer  │ (None, 30, 21)    │     10,773 │ dropout_2[0][0]   │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Drinks_Output_Layer │ (None, 30, 21)    │     10,773 │ dropout_2[0][0]   │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,718,890 (18.00 MB)

 Trainable params: 4,717,866 (18.00 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [5]:
# model1 = load_model('../models/shared_encoder_decoder02_noise.keras')

model.load_weights('../checkpoints/model_noise_06-0.28.weights.h5')



/media/mohamed/AC6030326030059C/CMP1Materials/Forth/First/NLP/Project/dataset/venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [6]:
model2 = load_model('../models/shared_encoder_decoder2.keras')

model2.summary()

Model: "Encoder_Decoder_NER"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_Sequence      │ (None, 30)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Embedding_Layer     │ (None, 30, 128)   │  3,906,816 │ Input_Sequence[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 30, 128)   │          0 │ Embedding_Layer[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Bidirectional_LSTM  │ [(None, 30, 256), │    263,168 │ dropout[0][0]     │
│ (Bidirectional)     │ (None, 128),      │            │                   │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ Bidirectional_LS… │
│ (Concatenate)       │                   │            │ Bidirectional_LS… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256)       │          0 │ Bidirectional_LS… │
│ (Concatenate)       │                   │            │ Bidirectional_LS… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Decoder_LSTM (LSTM) │ (None, 30, 256)   │    525,312 │ Bidirectional_LS… │
│                     │                   │            │ concatenate[0][0… │
│                     │                   │            │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Attention_Layer     │ (None, 30, 256)   │          0 │ Decoder_LSTM[0][… │
│ (Attention)         │                   │            │ Bidirectional_LS… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 30, 256)   │          0 │ Attention_Layer[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 30, 512)   │          0 │ Decoder_LSTM[0][… │
│ (Concatenate)       │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 30, 512)   │      2,048 │ concatenate_2[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 30, 512)   │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Order_Output_Layer  │ (None, 30, 5)     │      2,565 │ dropout_2[0][0]   │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,097,681 (53.78 MB)

 Trainable params: 4,698,885 (17.92 MB)

 Non-trainable params: 1,024 (4.00 KB)

 Optimizer params: 9,397,772 (35.85 MB)

In [7]:
input_ids_pizza_test = np.load('../data/processed/test/input_ids_pizza_test.npy')

padded_labels_pizza_test = np.load('../data/processed/test/padded_labels_pizza_test.npy')
padded_labels_drink_test = np.load('../data/processed/test/padded_labels_drink_test.npy')

# attention_masks_drink_test=np.load('data/processed/test/attention_masks_drink_test.npy', allow_pickle=True)
# attention_masks_pizza_test=np.load('data/processed/test/attention_masks_pizza_test.npy', allow_pickle=True)

padded_labels_test_second = np.load('../data/processed/test/padded_labels_test_second.npy')


In [8]:
processed_test_sentences = np.load(
    '../data/processed/test/processed_test_sentences.npy')

In [ ]:
X_test = {"input_ids": input_ids_pizza_test, "attention_mask_pizza": attention_masks_pizza_test, "attention_mask_drinks": attention_masks_drink_test}


In [9]:
predictions1 = np.array(model.predict(input_ids_pizza_test))

43/43 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step


In [10]:
predictions2 = np.array(model2.predict(input_ids_pizza_test))

43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step


In [8]:
predictions1.shape

(2, 1357, 30, 21)

In [11]:
entity_labels_drink=np.load('../data/processed/entity_labels_drink.npy')
entity_labels_pizza=np.load('../data/processed/entity_labels_pizza.npy')
enitity_labels_second=np.load('../data/processed/entity_labels_second.npy')

print(entity_labels_drink)
print(entity_labels_pizza)
print(enitity_labels_second)


['B-SIZE' 'I-SIZE' 'B-VOLUME' 'I-VOLUME' 'B-NOT_DRINKTYPE'
 'I-NOT_DRINKTYPE' 'B-NOT_NUMBER' 'I-NOT_NUMBER' 'B-NOT_CONTAINERTYPE'
 'I-NOT_CONTAINERTYPE' 'B-CONTAINERTYPE' 'I-CONTAINERTYPE' 'B-NOT_VOLUME'
 'I-NOT_VOLUME' 'B-NUMBER' 'I-NUMBER' 'B-DRINKTYPE' 'I-DRINKTYPE'
 'B-NOT_SIZE' 'I-NOT_SIZE']
['B-SIZE' 'I-SIZE' 'B-NOT_STYLE' 'I-NOT_STYLE' 'B-STYLE' 'I-STYLE'
 'B-NOT_NUMBER' 'I-NOT_NUMBER' 'B-QUANTITY' 'I-QUANTITY' 'B-NOT_TOPPING'
 'I-NOT_TOPPING' 'B-NUMBER' 'I-NUMBER' 'B-TOPPING' 'I-TOPPING'
 'B-NOT_QUANTITY' 'I-NOT_QUANTITY' 'B-NOT_SIZE' 'I-NOT_SIZE']
['B-PIZZAORDER' 'I-PIZZAORDER' 'B-DRINKORDER' 'I-DRINKORDER']


In [12]:
entities_id_drink = {i+1: str(e) for i, e in enumerate(entity_labels_drink)}
entities_id_drink[0] = 'O'
entities_id_drink

{1: 'B-SIZE',
 2: 'I-SIZE',
 3: 'B-VOLUME',
 4: 'I-VOLUME',
 5: 'B-NOT_DRINKTYPE',
 6: 'I-NOT_DRINKTYPE',
 7: 'B-NOT_NUMBER',
 8: 'I-NOT_NUMBER',
 9: 'B-NOT_CONTAINERTYPE',
 10: 'I-NOT_CONTAINERTYPE',
 11: 'B-CONTAINERTYPE',
 12: 'I-CONTAINERTYPE',
 13: 'B-NOT_VOLUME',
 14: 'I-NOT_VOLUME',
 15: 'B-NUMBER',
 16: 'I-NUMBER',
 17: 'B-DRINKTYPE',
 18: 'I-DRINKTYPE',
 19: 'B-NOT_SIZE',
 20: 'I-NOT_SIZE',
 0: 'O'}

In [13]:
entities_id_pizza = {i+1: str(e) for i, e in enumerate(entity_labels_pizza)}
entities_id_pizza[0] = 'O'
entities_id_pizza

{1: 'B-SIZE',
 2: 'I-SIZE',
 3: 'B-NOT_STYLE',
 4: 'I-NOT_STYLE',
 5: 'B-STYLE',
 6: 'I-STYLE',
 7: 'B-NOT_NUMBER',
 8: 'I-NOT_NUMBER',
 9: 'B-QUANTITY',
 10: 'I-QUANTITY',
 11: 'B-NOT_TOPPING',
 12: 'I-NOT_TOPPING',
 13: 'B-NUMBER',
 14: 'I-NUMBER',
 15: 'B-TOPPING',
 16: 'I-TOPPING',
 17: 'B-NOT_QUANTITY',
 18: 'I-NOT_QUANTITY',
 19: 'B-NOT_SIZE',
 20: 'I-NOT_SIZE',
 0: 'O'}

In [14]:
enitities_id_second = {i+1: str(e) for i, e in enumerate(enitity_labels_second)}
enitities_id_second[0] = 'O'
enitities_id_second

{1: 'B-PIZZAORDER',
 2: 'I-PIZZAORDER',
 3: 'B-DRINKORDER',
 4: 'I-DRINKORDER',
 0: 'O'}

In [15]:
def get_prediction_entities1(processed_sentences, predictions, entities_id):
    pred_entities = []
    for i in range(predictions.shape[0]):
        sen=[entities_id[np.argmax(predictions[i][j])] for j in range(predictions.shape[1])]
        pred_entities.append(sen[1:len(processed_sentences[i].split())+1])
    return pred_entities

In [22]:
predictions2.shape

(1357, 30, 5)

In [16]:
entities_pizza = np.array(get_prediction_entities1(
    processed_test_sentences, predictions1[0], entities_id_pizza), dtype=object)
entities_drink = np.array(get_prediction_entities1(
    processed_test_sentences, predictions1[1], entities_id_drink), dtype=object)
entities_second = np.array(get_prediction_entities1(processed_test_sentences,predictions2, enitities_id_second),dtype=object)

In [17]:
def process_second_entities(entities_second):
    current_entity = 'O'
    processed_entities = []
    for entity in entities_second:
        if entity =='O' and current_entity == 'O':
            processed_entities.append('O')
        elif entity == 'O' and current_entity != 'O':
            if current_entity.startswith('B-'):
                current_entity = 'I-'+current_entity[2:]
            processed_entities.append(current_entity)
        elif entity.startswith('I-') and current_entity == 'O':
            current_entity = 'B-'+entity[2:]
            processed_entities.append(current_entity)
        else:
            current_entity = entity   
            processed_entities.append(current_entity) 
            
    return processed_entities


process_second_entities(entities_second[120])

['B-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER']

In [25]:
numb = 130

print(processed_test_sentences[numb])
print(entities_second[numb])
print(process_second_entities(entities_second[numb]))

get me a small mushroom and sausage pizza with no pineapple
['O', 'O', 'B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER']
['O', 'O', 'B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER']


In [17]:
len(processed_test_sentences[533].split())

33

In [18]:
len(entities_drink[533])

29

In [19]:
len(entities_pizza[533])

29

In [20]:
len(entities_second[533])

29

In [28]:
def get_JSON(words, predictions1, predictions2, predictions3):
    words = words.split()
    order = {"ORDER": {"PIZZAORDER": [], "DRINKORDER": []}}
    current_pizza = None
    current_drink = None

    for i in range(len(words)):
        if predictions1[i] == 'B-PIZZAORDER':
            if current_pizza:
                order["ORDER"]["PIZZAORDER"].append(current_pizza)
            current_pizza = {"NUMBER": None, "SIZE": None, "STYLE": None, "AllTopping": []}
        elif predictions1[i] == 'B-DRINKORDER':
            if current_drink:
                order["ORDER"]["DRINKORDER"].append(current_drink)
            current_drink = {"NUMBER": None, "SIZE": None, "DRINKTYPE": None, "CONTAINERTYPE": None}

        if predictions2[i].startswith('B-'):
            tag = predictions2[i][2:]
            if tag == 'NUMBER':
                if current_pizza:
                    current_pizza["NUMBER"] = words[i]
                elif current_drink:
                    current_drink["NUMBER"] = words[i]
            elif tag == 'SIZE':
                if current_pizza:
                    current_pizza["SIZE"] = words[i]
                elif current_drink:
                    current_drink["SIZE"] = words[i]
            elif tag == 'TOPPING':
                current_pizza["AllTopping"].append({"NOT": False, "Quantity": None, "Topping": words[i]})

        if predictions3[i].startswith('B-'):
            tag = predictions3[i][2:]
            if tag == 'CONTAINERTYPE':
                current_drink["CONTAINERTYPE"] = words[i]
            elif tag == 'DRINKTYPE':
                current_drink["DRINKTYPE"] = words[i]
            elif tag == 'TOPPING':
                current_pizza["AllTopping"][-1]["Topping"] = words[i]

    if current_pizza:
        order["ORDER"]["PIZZAORDER"].append(current_pizza)
    if current_drink:
        order["ORDER"]["DRINKORDER"].append(current_drink)

    return order

# Example usage
input_sentence = "two large pizzas with ham and one bottle of diet coke and one small pizza with mushrooms"
model_1_prediction = ['B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'O', 'B-DRINKORDER', 'I-DRINKORDER',
                      'I-DRINKORDER', 'I-DRINKORDER', 'I-DRINKORDER', 'O', 'B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER']
model_2_prediction = ['B-NUMBER', 'B-SIZE', 'O', 'O', 'B-TOPPING', 'O', 'B-NUMBER', 'O', 'O', 'O', 'O', 'O', 'B-NUMBER', 'B-SIZE', 'O', 'O', 'B-TOPPING']
model_3_prediction = ['B-NUMBER', 'B-SIZE', 'O', 'O', 'O', 'O', 'B-NUMBER', 'B-CONTAINERTYPE',
                      'O', 'B-DRINKTYPE', 'I-DRINKTYPE', 'O', 'B-NUMBER', 'B-SIZE', 'O', 'O', 'O']

    

In [30]:
# import json

# formatted_order_json = json.dumps(get_JSON(
#     input_sentence, model_1_prediction, model_2_prediction, model_3_prediction), indent=4)
# print(formatted_order_json)

In [32]:
input_sentence = "two large pizzas with ham and one bottle of diet coke and one small pizza with mushrooms"
model_1_prediction = ['B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'O', 'B-DRINKORDER', 'I-DRINKORDER',
                      'I-DRINKORDER', 'I-DRINKORDER', 'I-DRINKORDER', 'O', 'B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER']
model_2_prediction = ['B-NUMBER', 'B-SIZE', 'O', 'O', 'B-TOPPING', 'O', 'B-NUMBER', 'O', 'O', 'O', 'O', 'O', 'B-NUMBER', 'B-SIZE', 'O', 'O', 'B-TOPPING']
model_3_prediction = ['B-NUMBER', 'B-SIZE', 'O', 'O', 'O', 'O', 'B-NUMBER', 'B-CONTAINERTYPE',
                      'O', 'B-DRINKTYPE', 'I-DRINKTYPE', 'O', 'B-NUMBER', 'B-SIZE', 'O', 'O', 'O']

# "(ORDER (PIZZAORDER (NUMBER two ) (SIZE large ) pizzas with (TOPPING ham ) ) and (DRINKORDER (NUMBER one ) (SIZE bottle ) of (DRINKTYPE diet coke ) ) and (PIZZAORDER (NUMBER one ) (SIZE small ) pizza with (TOPPING mushrooms ) ) )"
def format_to_TOP(input_sentence, model_1_prediction, model_2_prediction, model_3_prediction):
    words = input_sentence.split()
    result = "(ORDER "
    i = 0
    while i < len(words):
        if model_1_prediction[i].startswith('B-'):
            tag = model_1_prediction[i][2:]
            result += f"({tag} "
            while i < len(words) and (model_1_prediction[i].startswith('B-') or model_1_prediction[i].startswith('I-')):
                if model_2_prediction[i].startswith('B-'):
                    result += f"({model_2_prediction[i][2:]} {words[i]} ) "
                elif model_3_prediction[i].startswith('B-'):
                    result += f"({model_3_prediction[i][2:]} {words[i]} ) "
                else:
                    result += f"{words[i]} "
                i += 1
                if i < len(words) and model_1_prediction[i] == 'O':
                    break
            result = result.rstrip() + " ) "
        else:
            result += f"{words[i]} "
            i += 1
    result = result.rstrip() + " )"
    return result

formatted_TOP = format_to_TOP(input_sentence, model_1_prediction, model_2_prediction, model_3_prediction)
print(formatted_TOP)

"(ORDER i want (PIZZAORDER (NUMBER a ) pizza with (TOPPING pesto ) and (TOPPING mushrooms ) but no (NOT (TOPPING pineapple ) ) ) )"

input_sentence = "i want a pizza with pesto and mushrooms but no pineapple"
model_1_prediction = ['O', 'O', 'B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER' ]
model_2_prediction = ['O','O','B-NUMBER','O','B-TOPPING','O','B-TOPPING','O','O','O','B-NOT_TOPPING']
# model_3_prediction = ['O','O','O','O','B-TOPPING','O','B-TOPPING','O','O','O','B-NOT_TOPPING']

(ORDER (PIZZAORDER (NUMBER two ) (SIZE large ) pizzas with (TOPPING ham ) ) and (DRINKORDER (NUMBER one ) (CONTAINERTYPE bottle ) of (DRINKTYPE diet ) coke ) and (PIZZAORDER (NUMBER one ) (SIZE small ) pizza with (TOPPING mushrooms ) ) )


In [34]:
format_to_TOP(processed_test_sentences[3], entities_pizza[3], entities_drink[3], entities_second[3])

'(ORDER get me two pepsis a coke and (NUMBER (NUMBER five ) (SIZE large ) ) fantas )'

In [18]:
def get_TOP(words, predictions1, predictions2, predictions3):
    words=words.split()
    
    result = "(ORDER "
    i = 0
    not_flag = False
    c = min(len(words), len(predictions1), len(
        predictions2), len(predictions3))
    while i < c:
        if i < c and predictions3[i].startswith('B-'):
            tag = predictions3[i][2:]
            result += f"({tag} "
            while i < c and (predictions3[i].startswith('B-') or predictions3[i].startswith('I-')):
                if predictions1[i].startswith('B-'):
                    if(predictions1[i].startswith('B-NOT_')):
                        result += f"(NOT ({predictions1[i][6:]} {words[i]} ) "
                        not_flag = True
                    else:
                        result += f"({predictions1[i][2:]} {words[i]} ) "
                elif predictions2[i].startswith('B-'):
                    # Handle multi-word drink types
                    if i+1 < c and predictions2[i+1].startswith('I-'):
                        result += f"({predictions2[i][2:]} {words[i]} {words[i+1]} ) "
                        i += 1
                    else:
                        result += f"({predictions2[i][2:]} {words[i]} ) "
                else:
                    result += f"{words[i]} "
                i += 1
                if i < c and predictions3[i] == 'O':
                    break
            if not_flag:
                result = result.rstrip() + " ) ) "
                not_flag = False
            else:
                result = result.rstrip() + " ) "
        else:
            result += f"{words[i]} "
            i += 1
    result = result.rstrip() + " )"
    return result


get_TOP(processed_test_sentences[0], entities_pizza[0],
        entities_drink[0], entities_second[0])

'(ORDER i want (PIZZAORDER (NUMBER a ) pizza with (TOPPING pesto ) and (TOPPING mushrooms ) but no pineapple ) )'

In [39]:
print(entities_pizza[1])
print(processed_test_sentences[1])

['O', 'O', 'B-NUMBER', 'B-STYLE', 'O', 'B-QUANTITY', 'I-QUANTITY', 'B-TOPPING', 'O', 'O', 'B-QUANTITY', 'B-TOPPING', 'O', 'O', 'O']
i would like to try two medium tuna pizzas with extra cheese and no pesto


In [95]:
print(len(processed_test_sentences[0].split()))
print(len(entities_pizza[0]))

11
13


In [23]:
for i in range(len(processed_test_sentences)):
    lp=len(processed_test_sentences[i].split())
    p=len(entities_pizza[i])
    d=len(entities_drink[i])
    s=len(entities_second[i])
    if lp!=p or lp!=d or lp!=s:
        print(i)

533
634
713
753
950
973
1019
1034
1135
1169
1174
1262
1269


In [24]:
entities_drink[114]

['O', 'O', 'B-NUMBER', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [25]:
process_second_entities(entities_second[114])

['O',
 'O',
 'B-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER']

In [19]:
processed_entities_second = [process_second_entities(es) for es in entities_second]

In [58]:
for i in range(len(processed_entities_second)):
    if len(processed_entities_second[i])!=len(entities_second[i]):
        print(i)

In [55]:
len(processed_entities_second)

1357

In [20]:
tops = [get_TOP(processed_test_sentences[i], entities_pizza[i], entities_drink[i], processed_entities_second[i]) for i in range(len(processed_test_sentences))]

In [30]:
tops[14]

'(ORDER i (PIZZAORDER would like to have (NUMBER one ) pie along with (TOPPING ham ) and (TOPPING olives ) without pepperoni ) )'

In [135]:
import json
import numpy as np
from tqdm import tqdm

import sys
sys.path.append("..")

In [136]:
from helper_functions import extract_text, load_data, apply_negation, get_entity

In [137]:
TEST_PATH = '../data/raw/PIZZA_test.json'

In [138]:
_, explanations_test, topics_test, decoupled_topics_test = map(np.array, zip(
    *[extract_text(data, 'test') for data in load_data(TEST_PATH, 'test')]))

In [21]:
# np.save('../data/output/topics_test.npy', topics_test)
np.save('../data/output/tops_noise.npy', tops)

In [2]:
from entity_resolution import PizzaSkillEntityResolver
from semantic_matchers import is_unordered_exact_match, is_semantics_only_unordered_exact_match, \
    is_semantics_only_unordered_exact_match_post_ER, is_unordered_exact_match_post_ER, is_semantics_only_unordered_exact_match_post_ER_top_top
from trees import TopSemanticTree, ExpressSemanticTree

ModuleNotFoundError: No module named 'entity_resolution'